# Step 4: Analyze results

In [ ]:
import pycompwa.ui as pwa

In [ ]:
pwa.Logging('error')  # at this stage, we are not interested in the back-end
particle_list = pwa.read_particles('model.xml')
kinematics = pwa.create_helicity_kinematics('model.xml', particle_list)
kinematics.create_all_subsystems()
data_sample = pwa.read_root_data(input_file='generated_data.root')
phsp_sample = pwa.read_root_data(input_file='generated_phsp.root')
intensity_builder = pwa.IntensityBuilderXML(
    'model.xml', particle_list, kinematics, phsp_sample)
fit_result = pwa.load('fit_result.xml')
intensity = intensity_builder.create_intensity()
data_set = kinematics.convert(data_sample)
phsp_set = kinematics.convert(phsp_sample)
estimator, initial_parameters = \
    pwa.create_unbinned_log_likelihood_function_tree_estimator(
        intensity, data_set)
intensity.updateParametersFrom(fit_result.final_parameters)

## 4.1 Convert to pandas

It is easiest to analyze your data with [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). We are mostly interested in the distributions of the kinematic variables, so we first have to convert the `DataSet` instances to `DataFrame`s:

In [ ]:
import pandas as pd

In [ ]:
frame_data = pd.DataFrame(data_set.data)
frame_phsp = pd.DataFrame(phsp_set.data)
frame_data['weights'] = data_set.weights
frame_data

We added a weights column here as well, though in this case, all data weights are unity.

Also note that we converted the phase space data set. This is because the phase space serves as a uniform space over which to evaluate the intensity. We therefore attribute an intensity as a weight to each entry in the phase space `DataFrame`:

In [ ]:
intensity_set = intensity.evaluate(phsp_set.data)
frame_phsp['weights'] = intensity_set

## 4.2 Visualize

Now that we converted our data to [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html), we can use its full potential! For instance, we can easily bin the distributions into histograms.

### Kinematic variables

Currently, the columns of the `DataFrame`s created above contain final state IDs. Of course, you would rather see particle names there, but we need to use unique IDs in case there are identical particle names. To convert the number IDs to particle names in the eventual plots, we first extract a mapping from the `Kinematics` object:

In [ ]:
transition_info = kinematics.get_particle_state_transition_kinematics_info()
id_to_name = transition_info.get_final_state_id_to_name_mapping()


def replace_ids(title):
    """Replace all IDs in a string by particle names."""
    for id, name in id_to_name.items():
        title = title.replace(str(id), name)
    return title

Now you can apply the usual [matplotlib.pyplot](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.html) tools to plot the distributions.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_1d_comparison(name, bins=100, **kwargs):
    """Helper function for comparing the 1D distributions of fit and data."""
    frame_data[name].hist(bins=bins, density=True,
                          alpha=.5, label='data', **kwargs)
    frame_phsp[name].hist(bins=bins, weights=frame_phsp['weights'],
                          density=True, histtype='step', color='red',
                          label='fit', **kwargs)
    plt.ylabel('normalized intensity')
    title = replace_ids(name)
    plt.xlabel(title)

In [ ]:
plot_1d_comparison('theta_2_4_vs_3')
plt.legend()

Applying a transformation to one of the columns is also easily done for a `pandas.DataFrame`. For instance, if you are interested in the invariant mass (and not the **square value** of the invariant mass), simply do:

In [ ]:
from numpy import sqrt

frame_data['M(3,4)'] = sqrt(frame_data['mSq_(3,4)'])
frame_phsp['M(3,4)'] = sqrt(frame_phsp['mSq_(3,4)'])

plot_1d_comparison('M(3,4)')
plt.legend()

### Dalitz plots

Dalitz plots are 2-dimensional histograms of the square values of the invariant masses. We are therefore interested in the following variables:

In [ ]:
variable_names = data_set.data.keys()
[var for var in variable_names if var.startswith('mSq_')]

We now use [matplotlib.pyplot.hist2d](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist2d.html) to generate the Dalitz plots:

In [ ]:
def dalitz_plot(frame, mass_x, mass_y, bins=50, **kwargs):
    """Helper function to create a Dalitz plot with useful axis titles."""
    plt.hist2d(
        frame[mass_x],
        frame[mass_y],
        weights=frame['weights'],
        bins=bins,
        **kwargs,
    )
    plt.xlabel(replace_ids(mass_x))
    plt.ylabel(replace_ids(mass_y))

In [ ]:
from matplotlib.colors import LogNorm  # logarithmic z-axis

fig, axs = plt.subplots(1, 2, figsize=(9, 4))

plt.sca(axs[0])
axs[0].set_title('data')
dalitz_plot(frame_data, 'mSq_(3,4)', 'mSq_(2,4)', norm=LogNorm())

plt.sca(axs[1])
axs[1].set_title('fit')
dalitz_plot(frame_phsp, 'mSq_(3,4)', 'mSq_(2,4)', norm=LogNorm())

## 4.3 Calculate fit fractions

The original intensity model that we optimized consists of several components. All registered component names can be retrieved via ``get_all_component_names()``:

In [ ]:
intensity_builder.get_all_component_names()

As you can see, these components are amplitudes and intensities that are added coherently or incoherently.

In PWA, you are often interested in the fractions that those amplitudes contribute to the total intensity. These **fit fractions** can be calculated using the ``fit_fractions_with_propagated_errors()`` function. It requires amplitude or ``Intensity`` components that can be extracted from the ``IntensityBuilderXML`` instance. A nested list of the component names has to be specified as well. These are the names specified in the component XML attributes. If the inner lists contain more than one component, these components will be added coherently. In this way, you can calculate your own customized fit fractions.

In [ ]:
components = intensity_builder.create_intensity_components([
    ['coherent_J/psi_-1_to_gamma_-1+pi0_0+pi0_0'],
    ['J/psi_-1_to_f2(1270)_-1+gamma_-1;f2(1270)_-1_to_pi0_0+pi0_0;'],
    ['J/psi_-1_to_f2(1270)_-2+gamma_-1;f2(1270)_-2_to_pi0_0+pi0_0;'],
    ['J/psi_-1_to_f2(1270)_0+gamma_-1;f2(1270)_0_to_pi0_0+pi0_0;'],
    [
        'J/psi_-1_to_f2(1270)_-1+gamma_-1;f2(1270)_-1_to_pi0_0+pi0_0;',
        'J/psi_-1_to_f2(1270)_-2+gamma_-1;f2(1270)_-2_to_pi0_0+pi0_0;',
        'J/psi_-1_to_f2(1270)_0+gamma_-1;f2(1270)_0_to_pi0_0+pi0_0;',
    ],
])

The last step is to call the actual fit fraction calculation function ``fit_fractions_with_propagated_errors()``. Here, the first required argument is a list of component pairs. Each pair resembles the nominator and denominator of a fit fraction.

In [ ]:
fit_fractions = pwa.fit_fractions_with_propagated_errors([
    (components[1], components[0]),
    (components[2], components[0]),
    (components[3], components[0]),
    (components[4], components[0]),
], phsp_set, fit_result)

In [ ]:
for fraction in fit_fractions:
    print(fraction.name.replace(';', '\n  '))
    print(fraction.value, '+/-', fraction.error)
    print()

That's it! You can check some of the other examples to learn about more detailed features of ComPWA.

And, please, feel free to
[provide feedback](https://github.com/ComPWA/pycompwa/issues/new)
or [contribute](https://compwa.github.io/contribute.html) ;)